In [17]:
import numpy as np
from sklearn.datasets import make_spd_matrix
import math
import statsmodels.api as sm # for OLS 
from sklearn.ensemble import RandomForestRegressor # Our ML algorithm # Set up the environment 
randomseednumber = 11022018
np.random.seed(randomseednumber)

N = 500 # No. obs 
k=10 # = No. variables in x_i 
theta=0.5 # Structural parameter 
b= [1/k for k in range(1,11)] # x weights 
sigma = make_spd_matrix(k) # 
# NUmber of simulations 
MC_no = 50
def g(x):
    return np.power(np.sin(x),2)
def m(x,nu=0.,gamma=1.):
    return 0.5/math.pi*(np.sinh(gamma))/(np.cosh(gamma)-np.cos(x-nu))
# Array of estimated thetas to store results 
theta_est = np.zeros(shape=[MC_no,3])

for i in range(MC_no):
    # Generate data: no. obs x no. variables in x_i     
    X = np.random.multivariate_normal(np.ones(k),sigma,size=[N,])
    G = g(np.dot(X,b))
    M = m(np.dot(X,b))
    D = M+np.random.standard_normal(size=[500,])
    Y = np.dot(theta,D)+G+np.random.standard_normal(size=[500,])
    #     # Now run the different methods     #     # OLS --------------------------------------------------     
    OLS = sm.OLS(Y,D)
    results = OLS.fit()
    theta_est[i][0] = results.params[0]

    # Naive double machine Learning ------------------------     
    naiveDMLg =RandomForestRegressor(max_depth=2)
    # Compute ghat     
    naiveDMLg.fit(X,Y)
    Ghat = naiveDMLg.predict(X)
    naiveDMLm =RandomForestRegressor(max_depth=2)
    naiveDMLm.fit(X,D)
    Mhat = naiveDMLm.predict(X)
    # vhat as residual     
    Vhat = D-Mhat
    theta_est[i][1] = np.mean(np.dot(Vhat,Y-Ghat))/np.mean(np.dot(Vhat,D))

    # Cross-fitting DML -----------------------------------     # Split the sample     
    I = np.random.choice(N,np.int(N/2),replace=False)
    I_C = [x for x in np.arange(N) if x not in I]
    # Ghat for both     
    Ghat_1 = RandomForestRegressor(max_depth=2).fit(X[I],Y[I]).predict(X[I_C])
    Ghat_2 = RandomForestRegressor(max_depth=2).fit(X[I_C],Y[I_C]).predict(X[I])
    # Mhat and vhat for both     
    Mhat_1 = RandomForestRegressor(max_depth=2).fit(X[I],D[I]).predict(X[I_C])
    Mhat_2 = RandomForestRegressor(max_depth=2).fit(X[I_C],D[I_C]).predict(X[I])
    Vhat_1 = D[I_C]-Mhat_1
    Vhat_2 = D[I] - Mhat_2
    theta_1 = np.mean(np.dot(Vhat_1,(Y[I_C]-Ghat_1)))/np.mean(np.dot(Vhat_1,D[I_C]))
    theta_2 = np.mean(np.dot(Vhat_2,(Y[I]-Ghat_2)))/np.mean(np.dot(Vhat_2,D[I]))
    theta_est[i][2] = 0.5*(theta_1+theta_2)

C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_36372/2371280822.py:48: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  I = np.random.choice(N,np.int(N/2),replace=False)
C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_36372/2371280822.py:48: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, 

C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_36372/2371280822.py:48: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  I = np.random.choice(N,np.int(N/2),replace=False)
C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_36372/2371280822.py:48: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, 

C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_36372/2371280822.py:48: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  I = np.random.choice(N,np.int(N/2),replace=False)
C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_36372/2371280822.py:48: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, 

C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_36372/2371280822.py:48: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  I = np.random.choice(N,np.int(N/2),replace=False)
C:\Users\ALEXAN~1\AppData\Local\Temp/ipykernel_36372/2371280822.py:48: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, 

In [9]:
make_spd_matrix(k)

array([[ 1.0466168 ,  0.88645686, -0.25986783,  0.57789937,  0.41967425,
         0.45989046, -1.22094264, -0.0942414 ,  0.1528691 ,  0.69376707],
       [ 0.88645686,  2.82777113, -0.83572674,  1.28071398,  0.79965205,
         1.20671708, -3.0266778 , -0.2427946 ,  0.31578116,  1.85592132],
       [-0.25986783, -0.83572674,  1.04146113, -0.55293398, -0.36617277,
        -0.46862448,  1.28348747,  0.20860055, -0.08254574, -0.64401355],
       [ 0.57789937,  1.28071398, -0.55293398,  1.16906994,  0.54952489,
         0.48800919, -1.53592682, -0.01757595,  0.07425117,  0.95668605],
       [ 0.41967425,  0.79965205, -0.36617277,  0.54952489,  0.68554715,
         0.41052118, -1.16825359, -0.04406073,  0.25721656,  0.7968497 ],
       [ 0.45989046,  1.20671708, -0.46862448,  0.48800919,  0.41052118,
         1.14993817, -1.38209748, -0.01044797,  0.16696452,  0.82737287],
       [-1.22094264, -3.0266778 ,  1.28348747, -1.53592682, -1.16825359,
        -1.38209748,  4.82609831,  0.26733149